In [3]:
import pandas as pd
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter
import matplotlib.pyplot as plt
import numpy as np
import cv2

from IPython.display import HTML

In [4]:
# Names of the one-run files : 
eval_data = "performance_one-run-avg_color_hard_eval.csv"
pad_data = "performance_one-run-avg_color_hard_pad.csv"

# Name of the experiment videos
pad_video = "color_hard_pad_0_avg"
eval_video = "color_hard_eval_0_avg"

In [5]:
df_eval = pd.read_csv(eval_data, index_col=0)
df_pad = pd.read_csv(pad_data, index_col=0)

In [6]:
# Compute cumsum
df_eval['episode_0_cumul'] = df_eval['episode_0_reward'].cumsum()
df_pad['episode_0_cumul'] = df_pad['episode_0_reward'].cumsum()

In [92]:
def create_single_graph(dfs, names, out, mode="cumul", interval = 25 / 600 * 1000) :
    """
    Compute a reward graph 
    Args : - dfs : a list of dataframes of same length from which we want to plot the reward, 
           - names : name of each dataframe
           - out : graph video name (don't forget extension .mp4)
           - mode : either reward (plot avg or current reward) or cumulative reward
           - interval : speed at which each frame is displayed
    """
    
    # Number of frames
    frames = dfs[0].shape[0]
    # Video writer
    metadata = dict(title='Wav Spectrogram', artist='Matplotlib', comment='')
    writer = animation.FFMpegWriter(fps=60, metadata=metadata, bitrate=3500)
    
    # Data
    data = [df["episode_0_"+mode] for df in dfs]
    # Max yscale for plotting
    max_y = max([df.max() for df in data])
    
    # Global values for plotting
    x, ys = [], { i : [] for i in range(len(dfs))}
    fig, ax = plt.subplots()
    
    # Callback function
    def animate(i):
        
        # At step i add one point per df
        pts = []
        for df in data :
            pts.append(df[i])
        x.append(i)
        
        # Append to global list 
        for y, pt in zip(ys.values(), pts) :
            y.append(pt)

        ax.clear()
        
        # Plot sequence so far
        for y, n in zip(ys.values(), names) : 
            ax.plot(x, y, label=n)
        
        # Set limits
        ax.set_xlim([0,frames])
        ax.set_ylim([0, max_y])
        ax.legend()
    
    # Build graph and save it
    dyn_graph = animation.FuncAnimation(fig, animate, frames = frames, interval = interval, repeat = False)
    dyn_graph.save(out, writer = writer)

In [93]:
create_single_graph([df_pad, df_eval], ['pad', 'eval'], "cumul_test.mp4")

In [79]:
def merge_videos(names) :
    
    """
    Merge experiment and graphs videos
    Args : - names : ordered list of video files names
    """
    
    # Initialize reader
    cap = [cv2.VideoCapture(i) for i in names]
    frames = [None] * len(names); # current frames
    ret = np.full(len(names), True) # Whether video has ended or not
    imgs = [None] * len(names) # current full image
    
    
    # Video writer
    out = None 
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    # While we can read from every video file
    while ret.all():

        # We read one frame from each video file
        for i,c in enumerate(cap):
            if c is not None:
                ret[i], frames[i] = c.read();
                
                # We reshape to fit to final video
                if ret[i] : 
                    h, w, c = frames[i].shape
                    if h != 448 :
                        frames[i] = cv2.resize(frames[i], (int(w * 448 / h), 448) )

        # We convert to RGB for convenienve
        for i,f in enumerate(frames):
            if ret[i]:
                imgs[i] = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
        
        # We concatenate horizontally
        imgs_h = cv2.hconcat(imgs)
        if out == None :
            out = cv2.VideoWriter('output_cum.mp4', fourcc, 20, (imgs_h.shape[1], imgs_h.shape[0]))
        out.write(imgs_h)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # We close the readers / writers
    for c in cap:
        if c is not None:
            c.release()

    out.release()
    
    

In [66]:
merge_videos(["color_hard_pad_0_avg.mp4", "color_hard_eval_0_avg.mp4", "cumul.mp4"])